In [5]:
import os
import sys
import json
import spotipy
import webbrowser
import spotipy.util as util
from json.decoder import JSONDecodeError
import pandas as pd
import numpy as np
from itertools import compress
import tkinter as tk

# from bokeh.io import output_notebook, show
# from bokeh.plotting import figure
# from bokeh.models import TextInput

In [17]:
%env SPOTIPY_CLIENT_ID=d7975b1684ab45b7a28e97c355229077
%env SPOTIPY_CLIENT_SECRET=1b34039592a9481ca59d511da9d471a5
%env SPOTIPY_REDIRECT_URI=http://google.com/

env: SPOTIPY_CLIENT_ID=d7975b1684ab45b7a28e97c355229077
env: SPOTIPY_CLIENT_SECRET=1b34039592a9481ca59d511da9d471a5
env: SPOTIPY_REDIRECT_URI=http://google.com/


In [18]:
def authorize():
    # get username
    username = sys.argv[1]

    # 22jcqps2wnphydtifbil43nfy?si=uS3xsIU5T2mw903owjCZHw

    # erase cache and ask for permissions
    try:
        token = util.prompt_for_user_token(username)
    except:
    #     os.remove(f".cache-{username}")
        token = util.prompt_for_user_token(username)

    # create spotipyObject
    spotObj = spotipy.Spotify(auth=token)
    return spotObj

In [5]:
# code to print out json data in nice format
#print(json.dumps(VARIABLE,sort_keys=True,indent=4))

In [1]:
def artist_search(spotObj):
    global current_artist
    global current_data
    
    user = spotObj.current_user()
    display_name = user['display_name'] # grab display name from json with key
    
    artist = search_field.get() # get artist search query
    # handle empty search query
    if artist == '':
        display.configure(state=tk.NORMAL)
        display.insert(tk.END, 'the search field is empty\n')
        display.update()
    else:
        # let user know we are searching...
        display.configure(state=tk.NORMAL)
        display.insert(tk.END, 'searching for artist...\n')
        display.update()
        display.configure(state=tk.DISABLED)
        
        # artist found
        search_results = spotObj.search(artist,limit=1,offset=0,type='artist')
        artist_info = search_results['artists']['items'][0]
        artist_name = artist_info['name']
        current_artist = artist_name
        artist_id = artist_info['id'] # use artist id to find albums
        display.configure(state=tk.NORMAL)
        display.delete("end-2l","end-1l")
    #     display.delete('11.0','12.0')
        display.insert(tk.END, 'press ADD to add this artist:\n' + artist_name + '\n')
        display.configure(state=tk.DISABLED)

        artist_releases = spotObj.artist_albums(artist_id)['items'] # all of artists releases 

        release_type = [release['album_type'] for release in artist_releases]
        album_indices = [_type == 'album' for _type in release_type]
        artist_albums = list(compress(artist_releases, album_indices)) # only albums

        unique_dates = [] # to prevent duplciates
        unique_names = [] # to prevent duplciates
        data = [] # list to store album info
        for album in artist_albums:
            album_date = album['release_date']
            album_name = album['name']

            # first ensure album is a unique entry 
            if album_date not in unique_dates and album_name not in unique_names:
                album_id = album['id']
                album_image = album['images'][0]['url']
                album_track_n = album['total_tracks']

                unique_dates.append(album_date) # add entry to prevent duplicates
                unique_names.append(album_name)

                # obtain track info
                album_tracks = spotObj.album_tracks(album_id)['items']

                track_infos = []
                for track in album_tracks:
                    track_name = track['name']
                    track_id = track['id']
                    track_fts = track['artists']
                    track_fts = [artist for i,artist in enumerate(track_fts) if i!=0] # gets all artists except main
                    track_fts = [artist.get('name') for artist in track_fts]
                    ms = track['duration_ms']
                    secs = round(ms / 1000)
                    mins = secs // 60
                    secs = secs % 60
                    track_dur = '%d:%d' % (mins,secs)
                    track_info = {'artist':artist_name,'name':track_name,'id':track_id,'features':track_fts,'feature_count':len(track_fts),'duration':track_dur}
                    track_infos.append(track_info)

                row = [artist_name,album_id,album_image,album_name,album_date,album_track_n,track_infos]
                data.append(row) # add row to data

        # store album releases in df
        current_data = pd.DataFrame(data,columns = ['artist','id','cover','name','release','track_count','track_info'])